### Ingestion del archivo "person.json"

In [0]:
dbutils.widgets.text("p_enviroment","")
v_enviroment = dbutils.widgets.get("p_enviroment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

#### Paso 1 - Leer el archivo JSON usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
name_schema = StructType(fields = [
    StructField("forename", StringType(), True),
    StructField("surname", StringType(), True)
])

In [0]:
person_schema = StructType(fields = [
    StructField("personId", IntegerType(), False),
    StructField("personName", name_schema, True)
])

In [0]:
person_df = spark.read.schema(person_schema) \
                .json(f"{bronze_folder_path}/{v_file_date}/person.json")

#### Paso 2 - Renombrar las columnas y añadir nuevas columnas
##### 1. "personId" renombrar a "person_id"
##### 2. Agregar las columnas "ingestion_date" y "enviroment"
##### 3. Agregarlas columnas "name" a partir de la concatenación de "forename" y "surname"

In [0]:
from pyspark.sql.functions import col, concat, current_timestamp, lit

In [0]:
df_person_wit_column = add_ingestion_date(person_df).withColumnRenamed("personId","person_id") \
                                .withColumn("enviroment",lit(v_enviroment)) \
                                .withColumn("name", concat(col("personName.forename"), lit(" "), col("personName.surname"))) \
                                .withColumn("file_date", lit(v_file_date))

#### Paso 3 - Eliminar las columnas no necesarias

In [0]:
final_person_df = df_person_wit_column.drop(col("personName"))

#### Paso 4 - Insertar el DF en formato Parquet en la carpeta Silver

In [0]:
#overwrite_partition(final_person_df,"movie_silver","person","file_date")

In [0]:
merge_condition = 'tgt.person_id = src.person_id AND tgt.file_date = src.file_date'
merge_delta_lake(final_person_df, "movie_silver", "person", silver_foler_path, merge_condition, "file_date")

In [0]:
%sql
SELECT file_date,COUNT(1) 
FROM movie_silver.person
GROUP BY file_date;

In [0]:
dbutils.notebook.exit("Success!")